In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from io import BytesIO
from PIL import Image
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from textblob import TextBlob


In [2]:
# cell 2
df = pd.read_csv('memes.csv')
df = df[['caption','img_url','ups','num_comments','category','subreddit','created_utc']]
df.dropna(subset=['caption'], inplace=True)
df.reset_index(drop=True, inplace=True)


In [3]:
# cell 3
df['caption'] = df['caption'].astype(str)
df['caption_length'] = df['caption'].apply(len)
df['word_count'] = df['caption'].apply(lambda x: len(x.split()))
df['sentiment'] = df['caption'].apply(lambda x: TextBlob(x).sentiment.polarity)


In [ ]:
# cell 4
def image_features_from_url(url, timeout=5):
    try:
        r = requests.get(url, timeout=timeout)
        img = Image.open(BytesIO(r.content)).convert('L')
        arr = np.array(img)
        return float(arr.mean()), float(arr.std())
    except:
        return np.nan, np.nan

bs = []
cs = []
for url in df['img_url']:
    b,c = image_features_from_url(url)
    bs.append(b)
    cs.append(c)

df['brightness'] = bs
df['contrast'] = cs
df['brightness'].fillna(df['brightness'].median(), inplace=True)
df['contrast'].fillna(df['contrast'].median(), inplace=True)
